## Libraries Used: 

In [2]:
%%capture

import pandas as pd
from pandas_profiling import ProfileReport
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
import numpy as np


## Read and Process Data:

In [3]:
%%capture
#Define function to read data and profile it.

def file_reader(fileX, htmlX):
    filex = pd.read_csv(fileX)
    filex_df = pd.DataFrame(filex)
    filex_profile =ProfileReport(filex_df)
    html_name = f"{htmlX}.html"
    filex_profile.to_file(html_name)
    return filex_df


In [4]:
%%capture

# Training Data
allstate = 'as_train.csv'
name1 = 'as_train'
train_data = file_reader(allstate, name1)

In [4]:
%%capture

# Testing Data
allstate_test = 'as_test.csv'
name2 = 'as_test'
test_data = file_reader(allstate_test,name2)

Summarize dataset:  99%|█████████▉| 362/365 [03:52<00:04,  1.57s/it, Detecting duplicates]      /Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/ydata_profiling/model/pandas/duplicates_pandas.py:37: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[duplicated_rows]
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/ydata_profiling/model/pandas/duplicates_pandas.py:37: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[duplicated_rows]
/Library/Frameworks/Python.framework/V

In [9]:
%%capture

#unuseful column found in train data so we drp it.
train_data = train_data.iloc[:, :132]
train_data.head()

,id,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,...,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,cont14,loss
0,1,A,B,A,B,A,A,A,A,B,...,0.718367,0.335060,0.30260,0.67135,0.83510,0.569745,0.594646,0.822493,0.714843,2213.18
1,2,A,B,A,A,A,A,A,A,B,...,0.438917,0.436585,0.60087,0.35127,0.43919,0.338312,0.366307,0.611431,0.304496,1283.60
2,5,A,B,A,A,B,A,A,A,B,...,0.289648,0.315545,0.27320,0.26076,0.32446,0.381398,0.373424,0.195709,0.774425,3005.09
3,10,B,B,A,B,A,A,A,A,B,...,0.440945,0.391128,0.31796,0.32128,0.44467,0.327915,0.321570,0.605077,0.602642,939.85
4,11,A,B,A,B,A,A,A,A,B,...,0.178193,0.247408,0.24564,0.22089,0.21230,0.204687,0.202213,0.246011,0.432606,2763.85


In [10]:
%%capture

#Sample Test Data
test_data.head()

,id,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,...,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,cont14
0,4,A,B,A,A,A,A,A,A,B,...,0.281143,0.466591,0.317681,0.61229,0.34365,0.38016,0.377724,0.369858,0.704052,0.392562
1,6,A,B,A,B,A,A,A,A,B,...,0.836443,0.482425,0.443760,0.71330,0.51890,0.60401,0.689039,0.675759,0.453468,0.208045
2,9,A,B,A,B,B,A,B,A,B,...,0.718531,0.212308,0.325779,0.29758,0.34365,0.30529,0.245410,0.241676,0.258586,0.297232
3,12,A,A,A,A,B,A,A,A,A,...,0.397069,0.369930,0.342355,0.40028,0.33237,0.31480,0.348867,0.341872,0.592264,0.555955
4,15,B,A,A,A,A,B,A,A,A,...,0.302678,0.398862,0.391833,0.23688,0.43731,0.50556,0.359572,0.352251,0.301535,0.825823


In [12]:
%%capture

#Making a copy of original data frame

claimDX = train_data.copy() #Train Data to use
claimTX = test_data.copy() #Test Data to use

## Data Encoding

In [13]:
%%capture

#creating the encoders

encoder1 = OrdinalEncoder() #Ordinal Encoder
encoder2 = OneHotEncoder() # One hot encoder

In [14]:
%%capture

#Encode Part of the train data from categorical to continous 
for column in  claimDX.columns[:109]:
    claimDX[column]= encoder1.fit_transform(claimDX[[column]])
    
#Encode Part of the test data from categorical to coninout
for column in claimTX.columns[:109]:
    claimTX[column]= encoder1.fit_transform(claimTX[[column]])


In [21]:
%%capture

#split Data from target
d_target = claimDX['loss']

In [22]:
%%capture

#Data without target
claimDX = claimDX.iloc[:, :131]

,id,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,...,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,cont14
0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.310061,0.718367,0.335060,0.30260,0.67135,0.83510,0.569745,0.594646,0.822493,0.714843
1,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.885834,0.438917,0.436585,0.60087,0.35127,0.43919,0.338312,0.366307,0.611431,0.304496
2,2.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.397069,0.289648,0.315545,0.27320,0.26076,0.32446,0.381398,0.373424,0.195709,0.774425
3,3.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.422268,0.440945,0.391128,0.31796,0.32128,0.44467,0.327915,0.321570,0.605077,0.602642
4,4.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.704268,0.178193,0.247408,0.24564,0.22089,0.21230,0.204687,0.202213,0.246011,0.432606


In [31]:
%%capture

#List of Categorical columns
col_list = ['cat109', 'cat110', 'cat111', 'cat112', 'cat113', 'cat114', 'cat115','cat116']

claimDX[col_list]=claimDX[col_list].astype('category')

In [76]:
%%capture

col_list = ['cat109', 'cat110', 'cat111', 'cat112', 'cat113', 'cat114', 'cat115','cat116']

claimTX[col_list]=claimTX[col_list].astype('category')

## Split Data

In [32]:
%%capture

#split data in train and validation

claimDX_train, claimDX_val = train_test_split(claimDX, test_size= 0.2, random_state=7)

In [44]:
%%capture

#target split
d_target_train, d_target_test = train_test_split(d_target, test_size=0.2, random_state=7)

##  Model Training

- LightGBM

In [46]:
%%capture


#Create DATASET for lgbm training

lgbm_Dataset = lgb.Dataset(label = d_target_train, data = claimDX_train, categorical_feature = col_list)

In [47]:
%%capture

#Define params for training the lgbm model
params = {

    'objective': 'regression',
    'num_leaves': 31,
    'learning_rate': 0.1,
    'n_estimators': 131,
    'max_depth': 5,
    'metric': 'mse',
    'boosting_type': 'gbdt',
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'min_child_samples': 20,
    'reg_alpha': 0.1,
    'reg_lambda': 0.1,
    'random_state': 42,
}

In [48]:
%%capture


#Traim our first model

lgbm_model_one = lgb.train(params, lgbm_Dataset, num_boost_round=100)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/lightgbm/engine.py:172: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.049331 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3948
[LightGBM] [Info] Number of data points in the train set: 155673, number of used features: 131
[LightGBM] [Info] Start training from score 3039.985509
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No furth

## Validation:

In [49]:
%%capture


# First Prediction

first_prediction = lgbm_model_one.predict(claimDX_val)

In [52]:
%%capture

pred_df = pd.DataFrame(first_prediction)
pred_df

,0
0,1081.991755
1,3639.198223
2,3193.005708
3,2412.077924
4,4053.862748
...,...
38914,2166.241021
38915,3634.826044
38916,1389.991482
38917,4943.297866


In [56]:
%%capture

#Evaluation Metrics:

mse_score = mean_squared_error(d_target_test, pred_df)
mse_score = round(mse_score, 2)

In [60]:
%%capture

#square root of the mse 
sqrt_mse = round(np.sqrt(mse_score),2)
sqrt_mse

1890.58

In [66]:
%%capture

pred_df.min()

0    203.844136
dtype: float64

In [75]:
%%capture

#Visualize Validation 
plt.scatter(d_target_test, pred_df, alpha=0.5)
plt.plot([min(d_target_test), max(d_target_test)], [min(d_target_test), max(d_target_test)], color='red', linestyle='--', lw=2)
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')
plt.title('Regression Line Plot')

plt.show()

## Test

In [77]:
%%capture

#Prediction
test_pred = lgbm_model_one.predict(claimTX)
test_result = round(pd.DataFrame(test_pred),2)

In [93]:
%%capture

common_indices = d_target_train.index.intersection(test_result.index)

y_true_common = d_target_train.loc[common_indices]
y_pred_common = test_result.loc[common_indices]

In [97]:
%%capture

#Evaluation Metric
res_mse= mean_squared_error(y_true_common,y_pred_common)
res_rmse = round(np.sqrt(res_mse),2)
res_rmse


3611.21

In [98]:
%%capture

#Vizualize Test

plt.scatter(y_true_common, y_pred_common, alpha=0.5)
plt.plot([min(y_true_common), max(y_true_common)], [min(y_true_common), max(y_true_common)], color='red', linestyle='--', lw=2)
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')
plt.title('Regression Line Plot')

plt.show()